# Transfer Öğrenme: Hazır Modelleri Kullanmak
a. Transfer Öğrenme Nedir?
Transfer Öğrenme (Transfer Learning), önceden büyük veri setlerinde (örneğin ImageNet) eğitilmiş bir modeli, yeni bir görev için yeniden kullanma tekniğidir.

### Popüler Hazır Modeller

- **VGG16 / VGG19**: Basit, anlaşılır yapı. İyi özellik çıkarır.
- **ResNet50**: Derin ağlar için atık bağlantılar (skip connections) kullanır.
- **MobileNet**: Mobil cihazlar için optimize edilmiştir, hafiftir.
- **EfficientNet**: Yüksek doğruluk, düşük hesaplama maliyeti.

In [9]:
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [10]:
base_dir = './cats_and_dogs_filtered'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir,
                     batch_size=16,
                     class_mode='binary',
                     target_size=(224,224))

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                     batch_size=16,
                     class_mode='binary',
                     target_size=(224,224))

base_model = VGG16(weights='imagenet',
include_top=False,input_shape=(224,224,3))


base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid')
])


model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//validation_generator.batch_size)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5


/Users/ibrahimediz/anaconda3/envs/ortamCV/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 268s 2s/step - accuracy: 0.7890 - loss: 0.8558 - val_accuracy: 0.9647 - val_loss: 0.1016
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 259s 2s/step - accuracy: 0.9020 - loss: 0.3148 - val_accuracy: 0.9728 - val_loss: 0.0796
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.9395 - loss: 0.2233 - val_accuracy: 0.9788 - val_loss: 0.0673
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9405 - loss: 0.1908 - val_accuracy: 0.9708 - val_loss: 0.0903
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9505 - loss: 0.1501 - val_accuracy: 0.9718 - val_loss: 0.0783


In [11]:
test_loss,test_acc = model.evaluate(validation_generator)
print('Test accuracy: ',test_acc)
print('Test loss: ',test_loss)


63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9720 - loss: 0.0777
Test accuracy:  0.972000002861023
Test loss:  0.07771778851747513


## Fine Tuning


In [12]:
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(1e-5),
loss="binary_crossentropy",metrics=["accuracy"])

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//validation_generator.batch_size)

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.9600 - loss: 0.1144 - val_accuracy: 0.9748 - val_loss: 0.0661
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.9645 - loss: 0.1015 - val_accuracy: 0.9859 - val_loss: 0.0373
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 279s 2s/step - accuracy: 0.9805 - loss: 0.0595 - val_accuracy: 0.9819 - val_loss: 0.0403
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.9765 - loss: 0.0653 - val_accuracy: 0.9768 - val_loss: 0.0655
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.9835 - loss: 0.0517 - val_accuracy: 0.9849 - val_loss: 0.0410
